In [3]:
from visualizer import calibration
#TODO: import from waveorder
import visual 

from PyQt5 import QtCore

from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
from glob import glob
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import napari
from napari import Viewer
import os
import tifffile

from skimage.registration import phase_cross_correlation
import cv2 as cv
from skimage import io

import cupy as cp 
import imagej

%gui qt 
%matplotlib inline
plt.style.use('dark_background')


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [51]:
# main_folder = r'D:\Test\20220429_M25_cElegans_no_Agar' # 5 Movement Across FOV
# main_folder = r'H:\Test\20220503_M25_cEle2' # Movements off the center
# main_folder = r'H:\Test\20220503_M25_moreCelegans10pm' #Great moving in WF and BF


### Analyzing the Paneuronal OH15 all taken at 40FPS
# main_folder = r'H:\Test\CElegansOH1_5' #3 In plane for first 5 seconds hten away.
# main_folder = r'H:\Test\20220504_M25_OH6' #4 In Planes and movement. Seems like BF was on
# main_folder = r'H:\Test\20220504_M25_OH7' #4  8bt Moevment in 3D
# main_folder = r'H:\Test\20220504_M25_OH8' # 5 Great moving 
# main_folder = r'H:\Test\20220504_M25_OH9' #5  Very Dynamic 
# main_folder = r'H:\Test\20220504_M25_OH10' #5  Best one so far. Great SNR and lots of moving 
# main_folder = r'H:\Test\20220504_M25_OH11' #5 3 worsm in FOV
# main_folder = r'H:\Test\20220504_M25_celegansOH_late1' #5 Moving Crazy
# main_folder = r'H:\Test\20220504_M25_celegansOH_late2' #4 not much moving
# main_folder = r'H:\Test\20220504_M25_celegansOH_late3' #3 worm is swimming in z
# main_folder = r'H:\Test\20220504_M25_celegansOH_late4' #3 huge readout noise :(
# main_folder = r'H:\Test\20220504_M25_celegansOH_late5' #5 accidental low BF with WL. Sample is bright

#Scope Parameters
# FOV = 50e-6
cam_px = 6.0e-6
totalmag = 15.75
px_size_img = cam_px/totalmag
zstep = 2e-6
z_scale = zstep/px_size_img

print(px_size_img)
print(zstep/cam_px)
print(z_scale)

3.8095238095238096e-07
0.3333333333333333
5.25


### Loading center planes for evaluation

In [53]:
%%time
#Load Only center Camera to see if good FOV and signal

center_planes = calibration.lazy_dask_stack(main_folder,num_cams=4,px_depth='uint8', height=608, width =808)

# center_planes = calibration.lazy_dask_stack(main_folder,num_cams=4,px_depth='uint16', height=608, width =808)

ValueError: Need array(s) to concatenate

In [54]:
napari.view_image(center_planes, name='center_planes',scale=[z_scale,1,1],multiscale=False)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 303.5, 403.5), zoom=0.5999999999999999, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 600.0, 1.0), (0.0, 26.25, 5.25), (0.0, 608.0, 1.0), (0.0, 808.0, 1.0)), current_step=(300, 2, 304, 404), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'center_planes' at 0x1be30883eb0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, s

### Loading Dataset

In [12]:
%%time

# stack = calibration.lazy_dask_stack(main_folder,num_cams=25,px_depth='uint8', height=600, width =960)

stack = calibration.lazy_dask_stack(main_folder,num_cams=25,px_depth='uint16', height=608, width =808)

ValueError: cannot reshape array of size 245760 into shape (1,608,808)

### Align Datasets with Known Offsets


In [7]:
# Load offsets from desired folder 
main_folder_offset = r'H:\Test\20220503_M25_1um_yg'
offsets_file_load = os.path.join(main_folder_offset,"offsets.csv")
with open(offsets_file_load) as file_name:
    offsets = np.loadtxt(file_name, delimiter=",")

In [8]:
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset

offset_coordinates = offsets
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

(25, 3)
(800, 25, 608, 808)


In [9]:
#Convert numpy to dask?
da_stack_align = da.from_array(stack_aligned)
da_stack_align[:,12,:,:]= da_stack_align[:,12,:,:]*1.3

print(da_stack_align)

dask.array<setitem, shape=(800, 25, 608, 808), dtype=uint16, chunksize=(100, 25, 76, 101), chunktype=numpy.ndarray>


In [10]:
## View partial planes
da_stack_align_partial = da.from_array(stack_aligned[:,5:22,:,:])
viewer.add_image(da_stack_align_partial, name='dask_align',scale=[z_scale/2,1,1], multiscale=False)

<Image layer 'dask_align' at 0x1ae9a31d970>

In [2]:
viewer= napari.Viewer()

In [12]:
viewer.add_image(da_stack_align, name='dask_align',scale=[z_scale,1,1], multiscale=False)

<Image layer 'dask_align [2]' at 0x1ae9a4ed580>

In [18]:
# # Align the files
# ij_stack2  = ij.py.to_java(np.array(da_stack_align,dtype='uint16'))
# ij.ui().show('PSF', ij_stack2)

### CROP PSF TO ANALYZE


In [15]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
print(crop_region)
print(template_box)
print(template_box.shape)

[[724.          13.         155.31981982 172.29279279]
 [724.          13.         155.31981982 550.52702703]
 [724.          13.         507.03153153 550.52702703]
 [724.          13.         507.03153153 172.29279279]]
[[155 172]
 [507 551]]
(2, 2)


In [16]:
# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
# np_mip_8bit =visual.im_bit_convert(np_mip,bit=8)
min_val, max_val = template_box
t,c = template[0,0:2]
print(t,c)
crop_img = da_stack_align[:,:, min_val[0]:max_val[0],min_val[1]:max_val[1]]
# crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
viewer.add_image(crop_img,name='crop')

724 13


<Image layer 'crop' at 0x21703319f10>

### Image J Part

In [4]:
def start_imagej():
    import imagej
    global ij
    ij = imagej.init( mode='interactive')
    # path = r'C:\Users\yoshi\Documents\fiji-win64\Fiji.app'
    # ij = imagej.init(path,mode='interactive')
    ij.ui().showUI()
    print(ij.getVersion())

QtCore.QTimer.singleShot(0, start_imagej)

2.3.0/1.53f


In [13]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(da_stack_align))
ij.ui().show('PSF', ij_stack2)

In [22]:
ij.WindowManager.getActiveWindow()
a =ij.WindowManager.getCurrentImage()
anp = ij.py.from_java(a)
print(anp.shape)
# anp2 = anp[:,:,:,np.newaxis]
# print(anp2.shape)
# anp2 = np.swapaxes(anp,1,4)
# anp2.shape

(296, 535, 535, 25)


In [24]:
anp2 = np.swapaxes(anp,1,3)
print(anp2.shape)
anp2_da = da.asarray(anp2)
viewer.add_image(anp2_da, name='crop',scale=[z_scale,1,1],multiscale=False)

(296, 25, 535, 535)


<Image layer 'crop [2]' at 0x1ae96a31cd0>

C:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-

Rendering frames...


  0%|          | 0/121 [00:00<?, ?it/s]C:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
  1%|      

#### Save TIFF FIles

In [21]:
import tifffile

filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)
print(filepath_save)
tifffile.imsave(filepath_save,anp2_da)

H:\Test\20220503_M25_moreCelegans10pm\crop_section.ome.tif


C:\Users\yoshi\AppData\Local\Temp\ipykernel_17792\34166168.py:6: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(filepath_save,anp2_da)


### Read TIFF FILE

In [4]:
viewer = napari.Viewer()

In [5]:
filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)

In [7]:
crop_tiff = tifffile.imread(filepath_save)
print(crop_tiff.shape)
print(type(crop_tiff))
da_crop_tiff = da.from_array(crop_tiff)
viewer.add_image(crop_tiff, name='crop',scale=[z_scale,1,1],contrast_limits=(0,2**8-1))

(250, 25, 288, 288)
<class 'numpy.ndarray'>


<Image layer 'crop' at 0x19786245cd0>

C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda

Rendering frames...


  0%|          | 0/61 [00:00<?, ?it/s]C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), 

Rendering frames...


  0%|          | 0/61 [00:00<?, ?it/s]C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), 

Rendering frames...


  0%|          | 0/61 [00:00<?, ?it/s]C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), 

c


In [8]:
from napari_animation import Animation
animation = Animation(viewer)
animation.animate('test.gif',canvas_only = False )

### Playing with napari-animation plugin


In [41]:
from napari_animation import AnimationWidget

animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')

ModuleNotFoundError: No module named 'napari_animation'